1. https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
2. https://en.wikipedia.org/wiki/Receiver_operating_characteristic

# Определение ССЗ

## Описание

ссылка на соревнование - https://mlbootcamp.ru/ru/round/12/sandbox/

Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

| Объективные признаки  | Результаты измерения                   | Субъективные признаки (0/1) |
|-----------------------|----------------------------------------|-----------------------------|
| Возраст (в днях)      | Артериальное давление верхнее и нижнее | Курение                     |
| Рост                  | Холестерин                             | Употребление Алкоголя       |
| Вес                   | Глюкоза                                | Физическая активность       |
| Пол                   |                                        |                             |

Все показатели даны на момент осмотра.

`target` - наличие сердечно-сосудистых заболеваний (ССЗ)

#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

#### Загрузка датасета

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [2]:
df = pd.read_csv('train_case2.csv', sep=';')
# Удалим 2 столбца
df = df.drop(['id', 'cholesterol', 'gluc'], axis=1)
df.head(3)

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,0,0,1,0
1,20228,1,156,85.0,140,90,0,0,1,1
2,18857,1,165,64.0,130,70,0,0,0,1


## Предобработка

In [6]:
df.describe()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,0.088129,0.053771,0.803729,0.499700
std,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.283484,0.225568,0.397179,0.500003
min,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,0.000000,0.000000,0.000000,0.000000
25%,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
50%,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
75%,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,0.000000,0.000000,1.000000,1.000000
max,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,1.000000,1.000000,1.000000,1.000000


#### Поиск пропусков

In [7]:
pd.DataFrame(df.isnull().sum()).T

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
0,0,0,0,0,0,0,0,0,0,0


In [8]:
(df['age']/365).describe()

count    70000.000000
mean        53.339358
std          6.759594
min         29.583562
25%         48.394521
50%         53.980822
75%         58.430137
max         64.967123
Name: age, dtype: float64

#### Аномалии `height` -> замена на медиану

In [9]:
df.loc[df['height'] > 220, 'height'] = df['height'].median()

#### Аномалии `ap_hi`

In [10]:
# замена знака
df.loc[df['ap_hi'] <= 0, 'ap_hi'] = np.abs(df['ap_hi'])

In [11]:
# уменьшение разрядов
df.loc[(df['ap_hi'] > 500) & (df['ap_hi'] < 10000), 'ap_hi'] = df['ap_hi'] / 10

In [12]:
# уменьшение разрядов
df.loc[(df['ap_hi'] > 10000), 'ap_hi'] = df['ap_hi'] / 100

In [13]:
# замена на медиану
df.loc[(df['ap_hi'] > 250), 'ap_hi'] = df['ap_hi'].median()

In [14]:
# увеличение разрядов
df.loc[(df['ap_hi'] < 50), 'ap_hi'] = df['ap_hi'] * 10

In [15]:
df.loc[(df['ap_hi'] < 50), 'ap_hi'] = df['ap_hi'] * 10

In [16]:
df['ap_hi'].describe()

count    70000.000000
mean       126.997107
std         17.114820
min         60.000000
25%        120.000000
50%        120.000000
75%        140.000000
max        240.000000
Name: ap_hi, dtype: float64

#### Аномалия `ap_lo`

In [17]:
#  -> замена знака
df.loc[df['ap_lo'] < 0, 'ap_lo'] = np.abs(df['ap_lo'])

In [18]:
# -> замена на медиану
df.loc[df['ap_lo'] == 0, 'ap_lo'] = np.median(df['ap_lo'])

In [19]:
df.loc[df['ap_lo'] > 200, 'ap_lo'] = df['ap_lo'] / 10

In [20]:
df.loc[df['ap_lo'] > 200, 'ap_lo'] = df['ap_lo'] / 10

In [21]:
df.loc[df['ap_lo'] > 200, 'ap_lo'] = df['ap_lo'] / 10

In [22]:
df.loc[df['ap_lo'] < 50, 'ap_lo'] = df['ap_lo'] * 10

In [23]:
df.loc[df['ap_lo'] < 50, 'ap_lo'] = df['ap_lo'] * 10

In [24]:
s = df['ap_lo'] > 150
df.loc[s, ['ap_hi', 'ap_lo']] = df.loc[s, ['ap_lo', 'ap_hi']].values

In [25]:
df['ap_lo'].describe()

count    70000.000000
mean        81.686803
std          9.961945
min         50.000000
25%         80.000000
50%         80.000000
75%         90.000000
max        240.000000
Name: ap_lo, dtype: float64

#### Преобразование `gender`

In [26]:
df.groupby('gender').mean()

,age,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
gender,,,,,,,,,
1,19510.124577,161.353745,72.565605,126.365133,81.197424,0.017856,0.025500,0.802021,0.496727
2,19392.097875,169.947895,77.257307,128.550544,82.597363,0.218880,0.106375,0.806906,0.505231


In [27]:
# Примем, что средний вес мужчин > среднего веса женщин
df['gender'] = df['gender'].map({1: 0, 
                                 2: 1})

In [28]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
0,18393,1,168,62.0,110.0,80.0,0,0,1,0
1,20228,0,156,85.0,140.0,90.0,0,0,1,1
2,18857,0,165,64.0,130.0,70.0,0,0,0,1
3,17623,1,169,82.0,150.0,100.0,0,0,1,1
4,17474,0,156,56.0,100.0,60.0,0,0,0,0


#### Разделим наши данные на тренировочную и тестовую выборки

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['cardio', ], axis=1),
    df['cardio'], 
    random_state=0)

К полям:
<!-- - `gender`, `cholesterol` применим OHE-кодирование -->
<!-- - `age`, `height`, `weight`, `ap_hi`, `ap_lo` - standardScaler -->
<!-- - `gluc`, `smoke`, `alco`, `active` - оставим пока как есть -->

Добавим классификатор и запустим кросс-валидацию

In [38]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [42]:
# classifier = RandomForestClassifier(n_estimators=200, random_state=42)
classifier = DecisionTreeClassifier(max_depth=5, random_state=42)
# classifier = XGBClassifier(random_state=42)

In [43]:
%%time

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7863046003341406+-0.0072098004566204495
Wall time: 2.04 s


In [44]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.402778, F-Score=0.729, Precision=0.660, Recall=0.813


In [45]:
X_test_2 = pd.DataFrame(
    {
        'age': 18,
        'gender': 1,  # 1 == man
        'height': 164,
        'weight': 54.0,
        'ap_hi': 200,
        'ap_lo': 80,
        'smoke': 0,
        'alco': 0,
        'active': 1
    },
    index=[80282])

In [46]:
classifier.predict(X_test_2)

array([1], dtype=int64)

In [47]:
classifier.predict_proba(X_test_2)

array([[0.13611111, 0.86388889]])

In [37]:
# import dill

# with open("model_rfc.dill", "wb") as f:
#     dill.dump(classifier, f)

In [48]:
import dill

with open("model_dtc.dill", "wb") as f:
    dill.dump(classifier, f)

In [49]:
import pickle 

with open('model_dtc.pkl', 'wb') as file:
    pickle.dump(classifier,file)

In [44]:
# загрузка модели

# with open(MODEL_FILE_PATH,'rb') as file:
#     model_load=pickle.load(file)

In [50]:
!pip freeze

absl-py==0.11.0
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.12
anaconda-project==0.8.3
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1596828585465/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///C:/ci/astroid_1592487315634/work
astropy==4.0.2
astunparse==1.6.3
async-generator==1.10
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1600298409949/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1596578164842/work
Babel==2.8.0
backcall==0.2.0
backports.functools-lru-cache==1.6.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
bcrypt @ file:///C:/ci/bcrypt_1597936263757/work
beautifulsoup4 @ file:///tmp/build/80754af9/beautifulsoup4_1601924105527/work
bitarray @ file:///C:/ci/bitarray_1598866981584/work
bkcharts==0.2
bleach @ file:///tmp/build/80754af9/bleach_1600439572647/work
bokeh @ file:///C:/ci/bokeh_1602773517800/work
boto==2.49.0
Bottleneck==1.3.2
bro